In [2]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [6]:
# importing data
ride_df=pd.read_csv('Trip_data.csv')


In [87]:
ride_df.drop('Unnamed: 0',axis=1,inplace=True)

,COLUMN_1,TRIP_ID,FARE_AMOUNT,TRIP_STARTTIME,ORIGIN_LONGITUDE,ORIGIN_LATITUDE,DEST_LONGITUDE,DEST_LATITUDE,PASSENGER_COUNT,CUSTOMER_ID,...,CUSTOMER_RATED_FOR_TRIP,TRIP_CANCELLATION,CANCELLED_BY,REASON,YEAR,MONTH,DATE_RW,HOUR,MINUTES,SECONDS
0,0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,C38941,...,5,0,0,0,2015,5,7,19,52,6
1,1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,C35861,...,5,0,0,0,2009,7,17,20,4,56
2,2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,C23933,...,2,0,0,0,2009,8,24,21,45,0
3,3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,C20815,...,4,0,0,0,2009,6,26,8,22,21
4,4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,C23933,...,5,0,0,0,2014,8,28,17,47,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,41888610,4.9,2011-06-05 16:36:00 UTC,-73.932240,40.860995,-73.928995,40.851152,3,C19492,...,2,0,0,0,2011,6,5,16,36,0
19996,19996,23058424,6.9,2011-06-05 14:22:37 UTC,-74.003543,40.732092,-73.996072,40.752730,1,C52940,...,3,0,0,0,2011,6,5,14,22,37
19997,19997,22184989,8.0,2012-09-19 21:00:00 UTC,-74.004062,40.747932,-73.986727,40.771712,5,C51399,...,3,0,0,0,2012,9,19,21,0,0
19998,19998,47421583,11.0,2014-02-04 14:05:00 UTC,-73.979607,40.759452,-73.988660,40.747752,2,C15722,...,3,0,0,0,2014,2,4,14,5,0


In [88]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the  distance between two points on a Earth."""


    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    # Radius of the Earth in kilometers 
    r = 6371

    # Calculate the distance
    distance = r * c
    return distance

In [89]:
#  DataFrame named 'ride_df' with columns 'latitude' and 'longitude'
ride_df['DISTANCE_KM'] = ride_df.apply(lambda row: haversine_distance(row['ORIGIN_LATITUDE'], row['ORIGIN_LONGITUDE'], row['DEST_LATITUDE'], row['DEST_LONGITUDE']), axis=1)


In [90]:
# Convert the 'datetime_column' to datetime format
ride_df['TRIP_STARTTIME'] = pd.to_datetime(ride_df['TRIP_STARTTIME'])

# Create new columns for date and time (without UTC)
ride_df['TRIP_STARTTIME'] = ride_df['TRIP_STARTTIME'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [91]:
ride_df=ride_df.sort_values(by='TRIP_STARTTIME')

In [92]:
ride_df=ride_df.reset_index(drop=True)

In [93]:
# Convert 'pickup_datetime' to datetime format (skip this step if already in datetime format)
ride_df['TRIP_STARTTIME'] = pd.to_datetime(ride_df['TRIP_STARTTIME'])

In [94]:
def remove_outliers_iqr(series,q3):
    """Removes outliers from a pandas Series using the interquartile range (IQR) method.

    Args:
        series: A pandas Series.
        q3 : 75 percentile.

    Returns:
        A pandas Series with outliers removed.
    """
    # Calculate the first and third quartiles (Q1 and Q3) of the Series
    q1 = series.quantile(0.25)
    q3 = series.quantile(q3)
    
    # Calculate the interquartile range (IQR)
    iqr = q3 - q1
    
    # Define the lower and upper bounds for outliers
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    
    # Remove values outside the lower and upper bounds
    series_outliers_removed = series[(series >= lower_bound) & (series <= upper_bound)]
    
    
    return series_outliers_removed


In [95]:
ride_df['FARE_AMOUNT']=remove_outliers_iqr(ride_df['FARE_AMOUNT'],0.90)

In [97]:
# fill nan by median
ride_df['FARE_AMOUNT'].fillna(ride_df['FARE_AMOUNT'].median(),inplace=True)

In [98]:
ride_df.loc[ride_df['DISTANCE_KM']>2000,'DISTANCE_KM']=ride_df['DISTANCE_KM'].quantile(0.75)

In [99]:
ride_df.loc[ride_df.DISTANCE_KM==0,['FARE_AMOUNT']]=0

In [100]:
ride_df.loc[(ride_df.DISTANCE_KM >80) & (ride_df.FARE_AMOUNT ),['FARE_AMOUNT']]=49

In [101]:
ride_df.loc[(ride_df.DISTANCE_KM < 1) & (ride_df.FARE_AMOUNT >30 ),['FARE_AMOUNT']] = ride_df.FARE_AMOUNT.quantile(0.25)

In [102]:
ride_df['DISTANCE_KM_RANGE']=pd.cut(ride_df.DISTANCE_KM,bins=[-1,2,4,8,127],labels=['0-2','3-4','long','very_long'])
ride_df['distance_km_range']=pd.cut(ride_df.distance_km,bins=[-1,2.0,6,15,127],labels=['0-2KM','2.1-6KM','6.1-15KM','Above 15KM'])

In [103]:
ride_df=ride_df.loc[~(ride_df.ORIGIN_LATITUDE <= -91)]

In [105]:
ride_df=ride_df.loc[ride_df['DISTANCE_KM'] < 100]

In [106]:
ride_df.reset_index(drop=True,inplace=True)

In [107]:
s=ride_df.loc[ride_df.DISTANCE_KM!=0].index.to_list()


In [109]:
ride_df['HOURS_RANGE']=pd.cut(ride_df.HOUR,bins=[-1,3,6,8,11,14,17,21,24],labels=['12:00AM-03:00AM','03:00AM-06:00AM','06:00AM-09:00AM','09:00AM-12:00PM','12:00PM-03:00PM','03:00PM-06:00PM','06:00PM-09:00PM','12:00AM-3:00AM'])
# pd.cut(driver_df.Age,bins=[26,33,40,45,60],labels=['Young','Adult','Elder','Old age'])

In [111]:
ride_df['FARE_AMOUNT_range']=pd.cut(ride_df.FARE_AMOUNT,bins=[-1,5,20,40,500],labels=['Inexpensive','Affordable','Economical','Expensive'])

In [114]:
s2=ride_df.groupby(['year','month'])['Trip_ID'].count().values

In [115]:
ride_df=ride_df.loc[~(ride_df.fare_amount < 0)]

,COLUMN_1,TRIP_ID,FARE_AMOUNT,TRIP_STARTTIME,ORIGIN_LONGITUDE,ORIGIN_LATITUDE,DEST_LONGITUDE,DEST_LATITUDE,PASSENGER_COUNT,CUSTOMER_ID,...,YEAR,MONTH,DATE_RW,HOUR,MINUTES,SECONDS,DISTANCE_KM,DISTANCE_KM_RANGE,HOURS_RANGE,FARE_AMOUNT_range
0,7628,8688426,10.6,2009-01-01 02:05:03,-73.956635,40.771254,-73.991528,40.749778,2,C23970,...,2009,1,1,2,5,3,3.786736,short,12:00AM-03:00AM,Affordable
1,4217,25687629,3.7,2009-01-01 12:18:00,-74.008845,40.708023,-74.010642,40.711428,1,C39725,...,2009,1,1,12,18,0,0.407792,very_short,12:00PM-03:00PM,Inexpensive
2,2200,52238368,9.3,2009-01-01 12:26:00,-73.945648,40.801802,-73.970920,40.762820,1,C51673,...,2009,1,1,12,26,0,4.828700,long,12:00PM-03:00PM,Affordable
3,10585,3587367,17.3,2009-01-01 17:25:00,-73.978850,40.761850,-74.007757,40.705293,1,C25439,...,2009,1,1,17,25,0,6.744038,long,03:00PM-06:00PM,Affordable
4,17069,1714664,6.5,2009-01-01 18:15:36,-73.986143,40.758396,-73.987727,40.739858,2,C36486,...,2009,1,1,18,15,36,2.065646,short,06:00PM-09:00PM,Affordable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,7734,41015040,8.5,2015-06-30 20:05:47,-73.955315,40.804562,-73.942322,40.828529,1,C20815,...,2015,6,30,20,5,47,2.880661,short,06:00PM-09:00PM,Affordable
19995,16240,20222977,0.0,2015-06-30 20:46:03,0.000000,0.000000,0.000000,0.000000,4,C31247,...,2015,6,30,20,46,3,0.000000,very_short,06:00PM-09:00PM,Inexpensive
19996,3500,32020814,6.0,2015-06-30 20:46:20,-73.937576,40.804520,-73.938606,40.812618,4,C17135,...,2015,6,30,20,46,20,0.904686,very_short,06:00PM-09:00PM,Affordable
19997,3230,23535251,9.5,2015-06-30 21:39:29,-73.982468,40.772266,-73.976784,40.790325,2,C18630,...,2015,6,30,21,39,29,2.064285,short,06:00PM-09:00PM,Affordable


In [ ]:
ride_df.reset_index(drop=True,inplace=True)

In [ ]:
ride_df['CALENDAR_DATE']=ride_df['date'].astype('string') + '/'+ ride_df.month.astype('string') + '/'+ ride_df.year.astype('string')

In [ ]:
ride_df.loc[ride_df.Region=='Pubs',['Region']]='Restaurants'

In [ ]:
ride_df.loc[ride_df.Region=='0.0',['Region']]='Market'

In [ ]:
ride_df.reset_index(inplace=True,drop=True)

In [ ]:
ride_df.to_csv('cdg_demo_trip_data_nb.csv',index=False)